In [ ]:
#
# Simple example of a Julia linear pogramming model
#
# Convert from notebook to julia program:
#   python3 nb_to_jl.py lp_claude_revised_simplex.ipynb
#
 
using LinearAlgebra
using SparseArrays
using Random
using ArgParse

const DEFAULT_USE_PRESOLVE = true 
const DEFAULT_USE_INTERIOR_POINT_METHOD = false
const DEFAULT_USE_SPARSE_MATRIX = false
const DEFAULT_VERBOSE = true

global options = Dict()

const COMMAND_LINE_OPTION_FILENAME = "filename"
const COMMAND_LINE_OPTION_USE_PRESOLVE = "presolve"
const COMMAND_LINE_OPTION_USE_SPARSE_MATRIX = "sparse"
const COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD = "interior"
const COMMAND_LINE_OPTION_VERBOSE = "verbose"

const OPTION_FILENAME = "filename"
const OPTION_USE_PRESOLVE = "use_presolve"
const OPTION_USE_SPARSE_MATRIX = "use_sparse_matrix"
const OPTION_USE_INTERIOR_POINT_METHOD = "use_interior_point_method"
const OPTION_VERBOSE = "verbose"

# https://en.wikipedia.org/wiki/Revised_simplex_method
const MPS_EXAMPLE::String = """
NAME          APPLIED_INTEGER_PROGRAMMING_9_7

OBJSENSE
 MAX

ROWS
 N  OBJ
 L  ROW1
 L  ROW2
 L  ROW3

COLUMNS
    X1        OBJ       4
    X1        ROW1      1
    X1        ROW2      2
    X1        ROW3     -3
    X2        OBJ       3
    X2        ROW1      2
    X2        ROW2     -1
    X2        ROW3      2
    X3        OBJ       1
    X3        ROW1      3
    X3        ROW2      2
    X3        ROW3      1
    X4        OBJ       7
    X4        ROW1      1
    X4        ROW2      2
    X4        ROW3     -1
    X5        OBJ       6
    X5        ROW1     -3
    X5        ROW2      1
    X5        ROW3      2

RHS
    RHS1      ROW1      9
    RHS1      ROW2     10
    RHS1      ROW3     11

BOUNDS
 LO BOUND1    X1        0
 LO BOUND1    X2        0
 LO BOUND1    X3        0
 LO BOUND1    X4        0
 LO BOUND1    X5        0

ENDATA
"""
#=
/opt/homebrew/Cellar/highs/1.7.2/bin/highs --solution_file ex97_highs.txt ex97.mps
Running HiGHS 1.7.2 (git hash: n/a): Copyright (c) 2024 HiGHS under MIT licence terms
LP   ex97 has 3 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 7e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 1e+01]
Presolving model
3 rows, 5 cols, 15 nonzeros  0s
3 rows, 5 cols, 15 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 5(-0); elements 15(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999978563e+01 Ph1: 3(11); Du: 5(21) 0s
          4     9.4000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  9.4000000000e+01
HiGHS run time      :          0.00

Model status
Optimal

# Primal solution values
Feasible
Objective 94
# Columns 5
X1 7
X2 10
X3 0
X4 0
X5 6
# Rows 3
ROW1 9
ROW2 10
ROW3 11

# Dual solution values
Feasible
# Columns 5
X1 0
X2 0
X3 -16.4285714285714
X4 -2.2380952380952
X5 0
# Rows 3
ROW1 1.4761904761905
ROW2 5.1904761904762
ROW3 2.6190476190476

# Basis
HiGHS v1
Valid
# Columns 5
1 1 0 0 1 
# Rows 3
2 2 2 
=#

In [ ]:
function is_running_in_notebook()
    # Check if running in VS Code's Jupyter notebook environment
    if (haskey(ENV, "VSCODE_PID") || haskey(ENV, "VSCODE_CWD"))
        return true
    # Check if running in a general Jupyter environment (including VS Code)
    elseif isdefined(Main, :IJulia) && Main.IJulia.inited
        return true
    else
        return false
    end
end

In [ ]:
function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--filename", "-f"
            help = "path to the problem file (mps format)"
            arg_type = String
            required = true
        "--interior", "-i"
            help = "use interior point method (LP only)"
            arg_type = Bool           
        "--min"
            help = "minimization of the objective function"
            action = :store_true            
        "--max"
            help = "maximization of the objective function"
            action = :store_true            
        "--presolve"
            help = "use presolve (default true)"
            arg_type = Bool          
        "--simplex"
            help = "use simplex method (default)"
            arg_type = Bool          
        "--sparse", "-s"
            help = "use sparce matrix representation"
            arg_type = Bool
        "--verbose", "-v"
            help = "verbose output"
            action = :store_true
    end

    return parse_args(s)
end

In [ ]:
# Define a struct to represent a Linear Programming problem
struct LPProblem
    is_minimize::Bool  # True if the objective is to minimize
    c::Vector{Float64}  # Objective function coefficients
    A::Matrix{Float64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    vars::Vector{String}  # Variable names
    constraint_types::Vector{Char}  # Constraint types
end

In [ ]:
function read_mps_from_string(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => Dict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name, row_name, value = words
            value = parse(Float64, value)
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = Dict()
            end
            sections["COLUMNS"][col_name][row_name] = value
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            bound_type, _, var_name, value = words
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = zeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Add bound constraints to A and b
    n_bound_constraints = count(x -> x > -Inf, lb) + count(x -> x < Inf, ub)
    A_with_bounds = zeros(n_constraints + n_bound_constraints, n_vars)
    b_with_bounds = zeros(n_constraints + n_bound_constraints)
    
    A_with_bounds[1:n_constraints, :] = A
    b_with_bounds[1:n_constraints] = b
    
    bound_constraint_index = n_constraints
    for i in 1:n_vars
        if lb[i] > -Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = lb[i]
            push!(constraint_types, 'G')
        end
        if ub[i] < Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = ub[i]
            push!(constraint_types, 'L')
        end
    end

    return LPProblem(is_minimize, c, A_with_bounds, b_with_bounds, vars, constraint_types)
end

In [ ]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, sparse(lp.A), lp.b
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] ≈ A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] ≈ -A[j, :] && abs(b[i] + b[j]) < eps && 
                   ((constraint_types[i] == '≤' && constraint_types[j] == '≥') || 
                    (constraint_types[i] == '≥' && constraint_types[j] == '≤'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) ≈ 1 && constraint_types[i] == '='
                val = b[i] / col[i]
                fixed_vars[vars[j]] = val
                keep_cols[j] = false
                b .-= val * col
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = isempty(fixed_vars) ? 0.0 : sum(c[j] * val for (j, val) in enumerate(lp.vars) if haskey(fixed_vars, val))

    if options[OPTION_VERBOSE]
        println("Presolve summary:")
        println("Original problem size: $(m) x $(n)")
        println("Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("Number of fixed variables: $(length(fixed_vars))")
        println("Objective adjustment: $obj_adjust")
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

In [ ]:
#=
Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.
=#

function revised_simplex(lp::LPProblem)
    c, A, b = lp.c, lp.A, lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("Objective function coefficients c: ", lp.c)
    println("Constraint matrix A: ", lp.A)
    println("Right-hand side b: ", lp.b)
    println("Variables: ", lp.vars) 
    println("Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A I(m)]  # Augment A with identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("Initial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ b  # Solve B * x_B = b for basic variables
        
        println("Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = zeros(n)
            x[B] = x_B
            println("Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ A[:, q]
        
        println("Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("New basis: ", B)
        println("New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function revised_simplex_sparse(lp::LPProblem)
    c, A, b = lp.c, sparse(lp.A), lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("Objective function coefficients c: ", lp.c)
    println("Constraint matrix A: ", lp.A)
    println("Right-hand side b: ", lp.b)
    println("Variables: ", lp.vars)
    println("Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A sparse(I, m, m)]  # Augment A with sparse identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("Initial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ Vector(b)  # Solve B * x_B = b for basic variables
        
        println("Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (Vector(c[B])' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = spzeros(n)
            x[B] = x_B
            println("Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return Array(x[1:length(lp.vars)]), obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ Vector(A[:, q])  # Convert sparse column to dense vector
        
        println("Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("New basis: ", B)
        println("New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function count_nonzeros(A)
    return count(!iszero, A)
end

In [ ]:
function scale_problem!(A, b, c)
    println("Entering scale_problem!")
    m, n = size(A)
    println("Matrix dimensions: $m x $n")
    
    row_scale = 1 ./ max.(1e-8, sqrt.(sum(abs2, A, dims=2)[:]))
    col_scale = 1 ./ max.(1e-8, sqrt.(sum(abs2, A, dims=1)[:]))
    
    println("Row scaling range: [$(minimum(row_scale)), $(maximum(row_scale))]")
    println("Column scaling range: [$(minimum(col_scale)), $(maximum(col_scale))]")
    
    A ./= row_scale
    A ./= col_scale'
    b ./= row_scale
    c ./= col_scale
    
    println("Scaled matrix range: [$(minimum(abs.(A))), $(maximum(abs.(A)))]")
    println("Scaled RHS range: [$(minimum(abs.(b))), $(maximum(abs.(b)))]")
    println("Scaled cost range: [$(minimum(abs.(c))), $(maximum(abs.(c)))]")
    
    println("Exiting scale_problem!")
    return row_scale, col_scale
end

In [ ]:
function mehrotra_predictor_corrector(A, b, c, x, y, s, mu)
    println("Entering mehrotra_predictor_corrector")
    m, n = size(A)
    println("Matrix dimensions: $m x $n")
    
    # Predictor step
    rx = A' * y + s - c
    ry = A * x - b
    rs = x .* s
    
    println("Residual norms: rx=$(norm(rx)), ry=$(norm(ry)), rs=$(norm(rs))")
    
    D = Diagonal(@. sqrt(max(x, 1e-16) / max(s, 1e-16)))
    M = [A * D * D * A' A * D;
         D * A' Diagonal(s)]
    rhs = [-ry; -rx]
    
    println("M dimensions: $(size(M))")
    println("rhs dimensions: $(size(rhs))")
    
    # Use QR factorization for better numerical stability
    F = qr(M)
    dz = F \ rhs
    
    dy_aff = dz[1:m]
    dx_aff = D * (D * (dz[m+1:end] - A' * dy_aff))
    ds_aff = -rx - A' * dy_aff
    
    println("Affine step norms: dx_aff=$(norm(dx_aff)), dy_aff=$(norm(dy_aff)), ds_aff=$(norm(ds_aff))")
    
    # Compute centering parameter
    alpha_aff_p = min(1.0, minimum(-x ./ dx_aff[dx_aff .< 0]))
    alpha_aff_d = min(1.0, minimum(-s ./ ds_aff[ds_aff .< 0]))
    mu_aff = dot(x + alpha_aff_p * dx_aff, s + alpha_aff_d * ds_aff) / n
    sigma = max(0, min(1, (mu_aff / mu)^3))
    
    println("Centering parameter: sigma=$sigma")
    
    # Corrector step
    rhs = [-ry; -rx - rs ./ max.(x, 1e-16) + sigma * mu ./ max.(x, 1e-16) - dx_aff .* ds_aff ./ max.(x, 1e-16)]
    dz = F \ rhs
    dy = dz[1:m]
    dx = D * (D * (dz[m+1:end] - A' * dy))
    ds = -rx - A' * dy
    
    println("Corrector step norms: dx=$(norm(dx)), dy=$(norm(dy)), ds=$(norm(ds))")
    
    println("Exiting mehrotra_predictor_corrector")
    return dx, dy, ds, sigma
end

In [ ]:
function highs_inspired_ipm(lp::LPProblem; max_iter=100, tol=1e-8)
    println("Entering highs_inspired_ipm")
    c, A, b = copy(lp.c), copy(lp.A), copy(lp.b)
    m, n = size(A)
    
    println("Problem dimensions: $m rows, $n columns")
    println("Nonzeros in A: $(count_nonzeros(A))")
    
    # Scale the problem
    row_scale, col_scale = scale_problem!(A, b, c)
    println("Scaling applied")
    
    # Initialize variables
    x = ones(n)
    y = zeros(m)
    s = ones(n)
    
    # Initial mu
    mu = dot(x, s) / n
    println("Initial mu: $mu")
    
    for k in 1:max_iter
        # Compute residuals
        primal_res = norm(A * x - b) / (1 + norm(b))
        dual_res = norm(A' * y + s - c) / (1 + norm(c))
        
        # Compute objective values
        primal_obj = dot(c, x)
        dual_obj = dot(b, y)
        
        println("Iter $k: P.res = $primal_res, D.res = $dual_res, P.obj = $primal_obj, D.obj = $dual_obj, mu = $mu")
        
        # Check for convergence
        if primal_res < tol && dual_res < tol && (primal_obj - dual_obj) / (1 + abs(primal_obj)) < tol
            println("Optimal solution found")
            return x .* col_scale, primal_obj
        end
        
        # Mehrotra predictor-corrector
        dx, dy, ds, sigma = mehrotra_predictor_corrector(A, b, c, x, y, s, mu)
        
        println("dx dimensions: $(size(dx)), x dimensions: $(size(x))")
        println("ds dimensions: $(size(ds)), s dimensions: $(size(s))")
        
        # Compute step sizes with safeguards
        function safe_step_size(current, delta)
            if length(current) != length(delta)
                error("Dimension mismatch: current=$(length(current)), delta=$(length(delta))")
            end
            ratios = -current ./ delta
            valid_ratios = filter(r -> isfinite(r) && r > 0, ratios)
            return isempty(valid_ratios) ? 0.9995 : min(0.9995, minimum(valid_ratios))
        end
        
        alpha_pri = safe_step_size(x, dx)
        alpha_dual = safe_step_size(s, ds)
        
        println("Step sizes: alpha_pri=$alpha_pri, alpha_dual=$alpha_dual")
        
        # Update variables
        x .+= alpha_pri .* dx
        y .+= alpha_dual .* dy
        s .+= alpha_dual .* ds
        
        # Update mu
        mu = dot(x, s) / n * sigma
        
        println("Updated mu: $mu")
        
        # Multiple centrality corrections (simplified version)
        for cc_iter in 1:2
            rx = c - A' * y - s
            ry = b - A * x
            rs = x .* s
            if norm(rs .- mu) < 0.1 * mu
                println("Centrality correction converged after $cc_iter iterations")
                break
            end
            dx_cc, dy_cc, ds_cc, _ = mehrotra_predictor_corrector(A, b, c, x, y, s, mu)
            alpha_cc = safe_step_size(x, dx_cc)
            x .+= 0.5 .* alpha_cc .* dx_cc
            y .+= 0.5 .* alpha_cc .* dy_cc
            s .+= 0.5 .* alpha_cc .* ds_cc
            println("Centrality correction step size: $alpha_cc")
        end
        
        # Ensure positivity
        x .= max.(x, 1e-8)
        s .= max.(s, 1e-8)
        
        println("Min x: $(minimum(x)), Min s: $(minimum(s))")
    end
    
    println("Maximum iterations reached without convergence")
    println("Exiting highs_inspired_ipm")
    return x .* col_scale, dot(c, x)
end

In [ ]:
# main program

if is_running_in_notebook()
    println("Running as a notebook")    
else
    println("Running as a script")
end

options[OPTION_USE_PRESOLVE] = DEFAULT_USE_PRESOLVE
options[OPTION_USE_SPARSE_MATRIX] = DEFAULT_USE_SPARSE_MATRIX
options[OPTION_USE_INTERIOR_POINT_METHOD] = DEFAULT_USE_INTERIOR_POINT_METHOD
options[OPTION_VERBOSE] = DEFAULT_VERBOSE

mps_string = MPS_EXAMPLE
if !is_running_in_notebook()
    parsed_args = parse_commandline()

    # Process command-line name
    if haskey(parsed_args, COMMAND_LINE_OPTION_FILENAME) && !isnothing(parsed_args[COMMAND_LINE_OPTION_FILENAME])
        options[OPTION_FILENAME] = parsed_args[COMMAND_LINE_OPTION_FILENAME]
        if isfile(options[OPTION_FILENAME])
            mps_string = return open(read, options[OPTION_FILENAME])
        else            
            error("File not found or could not read: '$(options[OPTION_FILENAME])'")
        end
    end    

    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_PRESOLVE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_PRESOLVE])
        options[OPTION_USE_PRESOLVE] = parsed_args[COMMAND_LINE_OPTION_USE_PRESOLVE]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_SPARSE_MATRIX) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX])
        options[OPTION_USE_SPARSE_MATRIX] = parsed_args[COMMAND_LINE_OPTION_USE_SPARSE_MATRIX]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD) && !isnothing(parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD])
        options[OPTION_USE_INTERIOR_POINT_METHOD] = parsed_args[COMMAND_LINE_OPTION_USE_INTERIOR_POINT_METHOD]
    end
    if haskey(parsed_args, COMMAND_LINE_OPTION_VERBOSE) && !isnothing(parsed_args[COMMAND_LINE_OPTION_VERBOSE])
        options[OPTION_VERBOSE] = true
    end
end

if options[OPTION_VERBOSE]
    println("Options:")
    println("  Use presolve: $(options[OPTION_USE_PRESOLVE])")
    println("  Use sparse matrix: $(options[OPTION_USE_SPARSE_MATRIX])")
    println("  Use interior point method: $(options[OPTION_USE_INTERIOR_POINT_METHOD])")
    println("  Verbose output: $(options[OPTION_VERBOSE])")
end

# Example usage
lp = read_mps_from_string(mps_string)

# Presolve the problem
if options["use_presolve"]
    lp, fixed_vars, obj_adjust = presolve(lp)

    println("\nFixed variables:")
    for (var, val) in fixed_vars
        println("$var = $val")
    end

    println("\nReduced problem:")
    println("Minimize: $(lp.is_minimize)")
    println("c = $(lp.c)")
    println("A = $(lp.A)")
    println("b = $(lp.b)")
    println("vars = $(lp.vars)")
    println("constraint_types = $(lp.constraint_types)")

end

# solve the LP problem

if options["use_interior_point_method"]
        println("\nSolving the reduced problem using the Interior Point Method with dense matrices")
        execution_time = @elapsed x, obj_value = homogeneous_self_dual_ipm(lp)    
else
    if options["use_interior_point_method"]
        println("\nSolving the reduced problem using the Revised Simplex Method with sparse matrices")
        execution_time = @elapsed x, obj_value = revised_simplex_sparse(lp)
    else
        println("\nSolving the reduced problem using the Revised Simplex Method with dense matrices")
        execution_time = @elapsed x, obj_value = revised_simplex(lp)
    end
end
println("\nExecution time: $execution_time seconds")

# show the solution

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")

# A = [3.0 1.0 -3.0 2.0 1.0;
#         2.0 2.0 1.0 -1.0 2.0;
#         1.0 -1.0 2.0 2.0 -3.0]
# b = [9.0, 10.0, 11.0]
# c = [-1.0, -7.0, -6.0, -3.0, -4.0]  # Negated for maximization
# vars = ["X3", "X4", "X5", "X2", "X1"]
# lp = LPProblem(false, c, A, b, vars, ['≤', '≤', '≤'])

# x, obj_value = highs_inspired_ipm(lp)

# println("\nSolution:")
# for (var, val) in zip(lp.vars, x)
#     println("$var = $val")
# end
# println("Objective value: ", -obj_value)  # Negated back for maximization